In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Access Folder Google Drive
import os
CURRENT_PATH = "/content/drive/My Drive/PKL Ganesha 79 Tim1/tugas1/datasets"
os.chdir(CURRENT_PATH)

In [ ]:
"""
Aldrin Rayhan Putra
Kamis, 08 Juni 2023
"""

import pandas as pd
import plotly.express as px

DATA_PRODUKSI_KAYU_OLAHAN = 'produksi_kayu_olahan.csv'

In [ ]:
### Read Data
produksi_kayu_olahan_df = pd.read_csv(DATA_PRODUKSI_KAYU_OLAHAN)

print(produksi_kayu_olahan_df.columns)

Index(['tahun', 'bulan', 'provinsi', 'jenis', 'volume'], dtype='object')


In [ ]:
### Data Cleaning
# Filter only 'tahun', 'bulan', 'jenis', 'value usd' columns
new_df = produksi_kayu_olahan_df.copy() 

# Change 'bulan' to its corresponding month number
new_df['bulan'] = new_df['bulan'].replace(['Januari', 'Februari', 'Maret', 'April', 'Mei', 'Juni', 'Juli', 'Agustus', 'September', 'Oktober', 'November', 'Desember'], [1,2,3,4,5,6,7,8,9,10,11,12])

# Combine 'bulan' and 'tahun' columns
new_df['waktu'] = new_df['bulan'].astype(str) + '-' + new_df['tahun'].astype(str)

# Format 'waktu' column to datetime, but don't include the day
new_df['waktu'] = pd.to_datetime(new_df['waktu'], format='%m-%Y')

# Drop 'bulan' and 'tahun' columns
new_df = new_df.drop(['bulan', 'tahun'], axis=1)

# Change 'jenis' column to category
new_df['jenis'] = new_df['jenis'].astype('category')

# Sort 'waktu' column
new_df = new_df.sort_values(by='waktu')

# Remove row where volume is 0
new_df = new_df[new_df['volume'] != 0]

new_df

,provinsi,jenis,volume,waktu
6670,Sulawesi Selatan,Kayu Gergajian,1202.07,2018-01-01
6977,Jawa Tengah,Kayu Lapis dan LVL,83142.99,2018-01-01
6978,Jawa Timur,Kayu Lapis dan LVL,86384.13,2018-01-01
6979,Kalimantan Barat,Kayu Lapis dan LVL,19548.85,2018-01-01
6980,Kalimantan Selatan,Kayu Lapis dan LVL,41061.59,2018-01-01
...,...,...,...,...
19,Kalimantan Selatan,Bare Core,137.23,2023-04-01
936,Papua Barat,Veneer,98.10,2023-04-01
935,Papua,Veneer,3452.32,2023-04-01
921,Bali,Veneer,263.34,2023-04-01


In [ ]:
"""
DATA PREPROCESSING

Tujuan  : Mendapatkan informasi mengenai jumlah semua produksi kayu olahan dari tahun 2018 sampai dengan 2023
Catatan :
    1. Data yang digunakan adalah data produksi kayu olahan dari tahun 2018 sampai dengan 2023
"""

ranking_volume = new_df.filter(['jenis', 'volume'])

ranking_volume = ranking_volume.groupby(['jenis']).sum().reset_index()

ranking_volume = ranking_volume.sort_values(by='volume', ascending=False)

ranking_volume['volume'] = ranking_volume['volume'].astype(int)

ranking_volume = ranking_volume.reset_index(drop=True)

ranking_volume.index += 1

ranking_volume['persentase'] = ranking_volume['volume'] / ranking_volume['volume'].sum() * 100

ranking_volume['persentase'] = ranking_volume['persentase'].round(2)

ranking_volume

###### UNTUK KEBUTUHAN PPT ######
ranking_volume_ppt = ranking_volume.copy()

# Format the volume using thousand separator
ranking_volume_ppt['volume'] = ranking_volume_ppt['volume'].apply(lambda x: "{:,}".format(x))

# Change 'produk' column to category
ranking_volume_ppt['jenis'] = ranking_volume_ppt['jenis'].astype('category')

# Add '%' to 'persentase' column
ranking_volume_ppt['persentase'] = ranking_volume_ppt['persentase'].astype(str) + '%'

# Change column name
ranking_volume_ppt = ranking_volume_ppt.rename(columns={'jenis': 'Jenis Produk Kayu Olahan', 'volume': 'Volume (m3)', 'persentase': 'Persentase'})

ranking_volume_ppt

,Jenis Produk Kayu Olahan,Volume (m3),Persentase
1,Chipwood,"193,616,435",68.14%
2,Pulp,"44,625,377",15.71%
3,Kayu Lapis dan LVL,"22,144,311",7.79%
4,Kayu Gergajian,"13,486,945",4.75%
5,Veneer,"7,430,827",2.62%
6,Blockboard,"994,756",0.35%
7,Bare Core,"847,108",0.3%
8,Wood Pellet,"690,206",0.24%
9,Particle Board,"222,757",0.08%
10,Palet Kayu,"47,361",0.02%


In [ ]:
"""
DATA VISUALIZATION

Tujuan  : Mendapatkan informasi dari provinsi mana saja yang menghasilkan 5 produk dengan volume tertinggi dalam rentang waktu tertentu
Catatan :
    1. Data yang digunakan adalah data produksi kayu olahan dari tahun 2018 sampai dengan 2023
    2. Rentang waktu dapat diubah sesuai kebutuhan dengan mengisinya pada query
"""

top_5_products = ranking_volume.head(5)['jenis'].to_list()
year_start = '2019'
year_end = '2023'

for product in top_5_products:
    result_df = new_df.query(f'jenis == "{product}"')
    result_df = result_df.filter(['waktu', 'provinsi', 'volume'])
    result_df = result_df.query(f'waktu >= "{year_start}-01-01" and waktu <= "{year_end}-05-01"')
    result_df = result_df.filter(['provinsi', 'volume'])
    result_df = result_df.groupby(['provinsi']).sum().reset_index()
    result_df = result_df.sort_values(by='volume', ascending=False)
    result_df = result_df.reset_index(drop=True)
    
    # Plot using pie chart, show the name of the product on every slice and hide the legend
    fig = px.treemap(result_df,
                        path=[px.Constant('Total semua provinsi'), 'provinsi'],
                        values='volume',
                        color='volume',
                        title=f'Produk Kayu Olahan {product}',
                        labels={'jenis':'Produk', 'volume':'Volume Produk (m3)'},
                        template='plotly_white')

    fig.show()

In [ ]:
"""
DATA VISUALIZATION

Tujuan  : Mendapatkan informasi jumlah produksi kayu olahan per produk dari tahun 2018 sampai dengan 2023
Catatan :
    1. Data yang digunakan adalah data produksi kayu olahan dari tahun 2018 sampai dengan 2023
"""

fig = px.line(new_df.filter(['jenis', 'volume', 'waktu']).groupby(['jenis', 'waktu']).sum().reset_index(),
              x='waktu',
              y='volume',
              color='jenis',
              title='Produksi Kayu Olahan Indonesia',
              labels={'waktu':'Waktu', 'volume':'Volume (m3)'},
              height=600,
              markers=True)

fig.show()

In [ ]:
"""
DATA PREPROCESSING

Tujuan  : Mendapatkan informasi mengenai persentase per provinsi dari total produksi kayu olahan Indonesia
Catatan :
    1. Data yang digunakan adalah data produksi kayu olahan dari tahun 2018 sampai dengan 2023
    2. Rentang waktu dapat diubah sesuai kebutuhan dengan mengisinya pada query
"""

# Filter the range of time
total_volume_per_provinsi = new_df.query('waktu >= "2018-01-01" and waktu <= "2023-12-01"')

# Filter only 'provinsi' and 'volume' columns
total_volume_per_provinsi = total_volume_per_provinsi.filter(['provinsi', 'volume'])

# Group by 'provinsi' and sum the 'volume'
total_volume_per_provinsi = total_volume_per_provinsi.groupby(['provinsi']).sum().reset_index()

# Sort 'volume' column
total_volume_per_provinsi = total_volume_per_provinsi.sort_values(by='volume', ascending=False)

# Change 'provinsi' column to category
total_volume_per_provinsi['provinsi'] = total_volume_per_provinsi['provinsi'].astype('category')

# Convert 'volume' column to int
total_volume_per_provinsi['volume'] = total_volume_per_provinsi['volume'].astype(int)

# Reset index
total_volume_per_provinsi = total_volume_per_provinsi.reset_index(drop=True)

# Start index from 1
total_volume_per_provinsi.index = total_volume_per_provinsi.index + 1

# Get the sum of all volume
total_volume = total_volume_per_provinsi['volume'].sum()

# Add new column 'persentase' and fill it with 0
total_volume_per_provinsi['persentase'] = 0

# Calculate the percentage of each provinsi's volume
for i in range(len(total_volume_per_provinsi) - 1):
    total_volume_per_provinsi.loc[i+1, 'persentase'] = (total_volume_per_provinsi.loc[i+1, 'volume'] / total_volume) * 100
    
# Change 'persentase' column to float
total_volume_per_provinsi['persentase'] = total_volume_per_provinsi['persentase'].astype(float)

# Round 'persentase' column to 2 decimal places
total_volume_per_provinsi['persentase'] = total_volume_per_provinsi['persentase'].round(2)

###### UNTUK KEBUTUHAN PPT ######
total_volume_per_provinsi_ppt = total_volume_per_provinsi.copy()

# Format the volume using thousand separator
total_volume_per_provinsi_ppt['volume'] = total_volume_per_provinsi_ppt['volume'].apply(lambda x: "{:,}".format(x))

# Change 'produk' column to category
total_volume_per_provinsi_ppt['provinsi'] = total_volume_per_provinsi_ppt['provinsi'].astype('category')

# Add '%' to 'persentase' column
total_volume_per_provinsi_ppt['persentase'] = total_volume_per_provinsi_ppt['persentase'].astype(str) + '%'

# Change column name
total_volume_per_provinsi_ppt = total_volume_per_provinsi_ppt.rename(columns={'provinsi': 'Provinsi', 'volume': 'Volume (m3)', 'persentase': 'Persentase'})

total_volume_per_provinsi_ppt

,Provinsi,Volume (m3),Persentase
1,Riau,"133,635,316",47.03%
2,Sumatera Selatan,"68,654,354",24.16%
3,Jambi,"24,695,955",8.69%
4,Jawa Timur,"15,614,352",5.5%
5,Jawa Tengah,"11,674,955",4.11%
6,Sumatera Utara,"9,766,833",3.44%
7,Kalimantan Tengah,"5,388,479",1.9%
8,Kalimantan Timur,"2,986,785",1.05%
9,Kalimantan Selatan,"2,887,539",1.02%
10,Papua,"1,854,407",0.65%


In [ ]:
"""
DATA VISUALIZATION

Tujuan  : Mendapatkan informasi mengenai seluruh jenis produk kayu olahan yang diproduksi per provinsinya
Catatan :
    1. Data yang digunakan adalah data produksi kayu olahan dari tahun 2018 sampai dengan 2023
    2. Rentang waktu dapat diubah sesuai kebutuhan dengan mengisinya pada query
    3. Provinsi yang digunakan hanyalah 5 provinsi dengan produksi terbanyak dari tahun 2018 sampai dengan 2023
"""

# Get the top 5 provinsi with the highest volume
top_5_provinsi = total_volume_per_provinsi.head(5)['provinsi'].tolist()
year_start = "2018"
year_end = "2023"

for provinsi in top_5_provinsi:
    # Filter 'provinsi' to the desired province
    jenis_per_provinsi = new_df.query('provinsi == @provinsi')

    # Filter 'waktu' to the desired range of time
    jenis_per_provinsi = jenis_per_provinsi.query(f'waktu >= "{year_start}-01-01" and waktu <= "{year_end}-12-01"')

    # Filter the datafrane to only needed columns
    jenis_per_provinsi = jenis_per_provinsi.filter(['jenis', 'volume'])

    # Group the dataframe by 'jenis' and sum the 'volume' column
    jenis_per_provinsi = jenis_per_provinsi.groupby(['jenis']).sum().reset_index()

    # Remove row where volume is 0
    jenis_per_provinsi = jenis_per_provinsi[jenis_per_provinsi['volume'] != 0]

    # Plot using pie chart, show the name of the product on every slice and hide the legend
    fig = px.pie(jenis_per_provinsi,
                    values='volume',
                    names='jenis',
                    title=f'Produksi Kayu Olahan {provinsi}',
                    labels={'jenis':'Jenis Produk', 'volume':'Volume (m3)'},
                    height=600,
                    color_discrete_sequence=px.colors.sequential.RdBu,
                    hole=0.5)

    fig.update_traces(textposition='inside', textinfo='percent+label', showlegend=False)

    fig.show()

    # Plot the dataframe using treemap - OPTIONAL
    # fig = px.treemap(jenis_per_provinsi,
    #                     path=[px.Constant('Total semua jenis produk'), 'jenis'],
    #                     values='volume',
    #                     color='volume',
    #                     title=f'Jenis-Jenis Produk Kayu Olahan {provinsi}',
    #                     labels={'jenis':'Jenis Produk', 'volume':'Volume (m3)'},
    #                     template='plotly_white')
    # fig.show()